In [8]:
# imports and setup
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

# USER_INPUT is noted where the user is required to edit for their own use.

final = r'USER_INPUT' # working data directory

In [2]:
# Define Winter DOY Function
def calculate_winter_doy(date):
    if pd.isna(date):  # Handle NaT values
        return None
    if date.month >= 10:  # From October to December (current year)
        return date.dayofyear - 273  # October 1 = 0 (non-leap year)
    else:  # From January to April (next year)
        return date.dayofyear + 92  # Add days from previous year's winter

In [4]:
# Prepare dataset
training_data = []
all_lake_data = {}  # Will store {lake_id: {winter_year: {week: weekly_afdd_value}}}
for f in tqdm([i for i in os.listdir(final)], desc='Building dataset...', unit='file'):
    fp = os.path.join(final, f)
    lake_data = pd.read_csv(fp)
    lake_data['date'] = pd.to_datetime(lake_data['date'])
    # Calculate winter DOY
    lake_data['winter_doy'] = lake_data['date'].apply(calculate_winter_doy)
    # Calculate winter year
    lake_data["winter_year"] = lake_data['date'].apply(
        lambda d: d.year if d.month >= 10 else d.year - 1)
    # Create ice binary variable (1 for ice, 0 for open water)
    lake_data['ice_binary'] = ((lake_data['ice_class'] == 'full') |  # pipe is OR
                               (lake_data['ice_class'] == 'partial')).astype(int)
    # Get lake properties
    lake_id = int(f.split('_')[1].split('.')[0])
    # Assign week number (relative to winter year) using winter DOY
    lake_data['winter_week'] = lake_data['winter_doy'] // 7
    # Calculate weekly temperature averages and AFDD
    lake_data['tmean_C'] = (lake_data['tmin'] + lake_data['tmax']) / 2
    lake_data['freezing_degrees'] = lake_data['tmean_C'].apply(lambda x: max(0 - x, 0))
    lake_data['afdd'] = lake_data.groupby('winter_year')['freezing_degrees'].cumsum()
    weekly_avg_afdd = lake_data.groupby(['winter_year', 'winter_week'])['afdd'].mean().reset_index()
    weekly_avg_afdd.rename(columns={'afdd': 'weekly_afdd'}, inplace=True)
    lake_data = pd.merge(lake_data, weekly_avg_afdd, on=['winter_year', 'winter_week'], how='left')
    weekly_avg_low = lake_data.groupby(['winter_year', 'winter_week'])['tmin'].mean().reset_index()
    weekly_avg_low.rename(columns={'tmin': 'weekly_mean_low'}, inplace=True)
    # Merge weekly AFDD back to the lake_data dataframe
    lake_data = lake_data.merge(weekly_avg_low, on=['winter_year', 'winter_week'], how='left')
    if lake_id not in all_lake_data: # at this point in the code, it won't be, so this conditional adds it in to the higher dictionary.
        all_lake_data[lake_id] = {}
    for _, row in lake_data.iterrows():
        year = row['winter_year']
        week = row['winter_week'] # starts at 0
        afdd7 = row['afdd'] # same
        wk_mean_low = row['weekly_mean_low'] # same
        if year not in all_lake_data[lake_id]:
            all_lake_data[lake_id][year] = {}
        all_lake_data[lake_id][year][week] = {
            'afdd': afdd7,
            'mean_low': wk_mean_low
        }
    # Only include rows with valid ice classification
    valid_rows = lake_data[~pd.isna(lake_data['ice_class'])].copy()
    # Add each valid row to the training data
    for _, row in valid_rows.iterrows():
        training_data.append({  # misleading name ....
            'elevation': row['Elevation_m'],
            'latitude': row['Latitude'],
            'afdd': row['afdd'],
            'area': row['Area_sqkm'],
            '$ T_{wk,L} $': row['weekly_mean_low'],
            'ice_binary': row['ice_binary'], # target
            'winter_year': row['winter_year'],
            'lake_id': lake_id,
            'winter_doy': row['winter_doy'],
        })
# Convert to dataframe
ice_df = pd.DataFrame(training_data)
# Handle any NaNs that might remain
ice_df = ice_df.dropna()


Building dataset...:   0%|          | 0/2872 [00:00<?, ?file/s]

### The following 2 cells were used to estimate the missing temperature data for Jan-Jun 2024. The Daymet temperature data set has since been updated to include these months, so this section is no longer necessary. However, it is retained here for reference.

In [5]:
# def estimate_2024_temp_values(lake_id, winter_doy, all_lake_data):
#     """Estimate temperature values for 2024 based on historical patterns, including AFDD."""
#     week = winter_doy // 7
#     values_low = []
#     values_afdd = []
#     # Loop over historical years for this lake
#     if lake_id in all_lake_data:
#         for year in all_lake_data[lake_id]:
#             if year < 2023 and week in all_lake_data[lake_id][year]:  # Only use past years
#                 values_low.append(all_lake_data[lake_id][year][week]['mean_low'])
#                 values_afdd.append(all_lake_data[lake_id][year][week]['afdd'])
#     # Return the mean, or np.nan if no values found
#     return (
#         float(np.mean(values_low)) if values_low else np.nan,
#         float(np.mean(values_afdd)) if values_afdd else np.nan
#     )

In [6]:
# def get_weekly_temp_values(lake_id, year, winter_doy, all_lake_data):
#     """
#     Get weekly temperature values for a given lake, year, and winter day.
#     Returns mean_low, and afdd, with a fallback to default or averaged values
#     if specific data is unavailable.
#     """
#     week = winter_doy // 7
#     # Try to get the value for the exact week
#     if lake_id in all_lake_data and year in all_lake_data[lake_id] and week in all_lake_data[lake_id][year]:
#         return (
#         all_lake_data[lake_id][year][week]['mean_low'],
#         all_lake_data[lake_id][year][week]['afdd']
#     )
#     else:
#         return None

## Create Complete Dataset for Analysis

In [7]:
# Create a complete date range for all lakes and winters
all_lakes = ice_df['lake_id'].unique()
all_winters = range(ice_df['winter_year'].min(), ice_df['winter_year'].max() + 1) # +1 because `range` doesn't include the last year by default
# Should now account for May-June
print("\nGenerating complete dataset with gap filling...")
# Get STATIC lake properties for each lake
lake_properties = ice_df.groupby('lake_id').agg({
    'elevation': 'first',
    'latitude': 'first',
    'area': 'first',
}).reset_index()

complete_grid = []  # Create empty list for full grid
winter_2023_lakes = set()  # Track which lake-years need special handling
for _, lake in tqdm(lake_properties.iterrows(), total=len(lake_properties), desc='Processing lakes'):
    for year in all_winters:
        # Get data for this lake and winter
        lake_year_data = ice_df[(ice_df['lake_id'] == lake['lake_id']) &
                                (ice_df['winter_year'] == year)]
        # Special handling for 2024
        is_winter_2023 = (year == 2023)
        if is_winter_2023:
            winter_2023_lakes.add(lake['lake_id'])
        if not lake_year_data.empty:
            # Create lookup dictionaries
            afdd_lookup = dict(zip(lake_year_data['winter_doy'], lake_year_data['afdd']))
            low_temp_lookup = dict(zip(lake_year_data['winter_doy'], lake_year_data['$ T_{wk,L} $']))
            ice_lookup = dict(zip(lake_year_data['winter_doy'], lake_year_data['ice_binary']))
            is_estimated = False
            for day in range(1, 272): # does not account for leap years. This establishes the rolling weeks.
                # For 2024, handle the Jan-Jun gap (winter_doy >= 92)
                if is_winter_2023 and day >= 92: # This is the IS DOY
                    # Use estimates for 2024 based on climatological averages
                    low_temp, afdd_value = estimate_2024_temp_values(
                        lake['lake_id'], day, all_lake_data
                    )
                    is_estimated = True
                else:
                    # Get values from lookups or calculate from weekly averages
                    afdd_value = afdd_lookup.get(day)
                    low_temp = low_temp_lookup.get(day)

                    # If missing, get from weekly data...
                    if low_temp is None:
                        low_temp, afdd_value = get_weekly_temp_values(lake['lake_id'], year, day, all_lake_data)
                # Add entry to complete grid
                complete_grid.append({
                    'lake_id': lake['lake_id'],
                    'winter_year': year,
                    'winter_doy': day, #
                    'elevation': lake['elevation'],
                    'latitude': lake['latitude'],
                    'area': lake['area'],
                    '$ T_{wk,L} $': low_temp,
                    'afdd': afdd_value,
                    'is_2024_estimate': 1 if is_estimated else 0,
                    'data_type': 'Observed' if day in ice_lookup else 'Modeled',
                    'ice_binary': ice_lookup.get(day)  # Will be None for days we need to predict
                })

# Convert to dataframe
complete_df = pd.DataFrame(complete_grid)
# Print info about the 2024 gap
print(f"\nFilled data for Jan-Apr 2024 for {len(winter_2023_lakes)} lakes using climatological averages")


Generating complete dataset with gap filling...


Processing lakes:   0%|          | 0/2819 [00:00<?, ?it/s]


Filled data for Jan-Apr 2024 for 2819 lakes using climatological averages


## Model Training and Testing: Cross-Validation

In [13]:
# Cross-validation cell
#################### THIS IS THE FEATURE SET ###############################
X = ice_df[['elevation', 'latitude', 'area', 'afdd', '$ T_{wk,L} $']]
############################################################################
y = ice_df['ice_binary'] # Target

In [14]:
# Use lake_id as the grouping variable for cross-validation
group_kfold = GroupKFold(n_splits=5) # it was found that increasing fold numbers did not increase performance and increased computational cost. This is why we are using 5 folds.
cv_scores = []
cv_auc_scores = []
cv_confusion_matrices = []

print("\nPerforming lake-based cross-validation...")
for i, (train_idx, test_idx) in enumerate(group_kfold.split(X, y, groups=ice_df['lake_id'])):
    # Split data
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    # Standardize
    scaler = StandardScaler() # NEED THIS INSTANCE
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    # Train model
    log_reg = LogisticRegression(max_iter=100, random_state=42, verbose = 1) # default solver lbfgs
    log_reg.fit(X_train_scaled, y_train)
    # Test model
    y_pred = log_reg.predict(X_test_scaled)
    y_prob = log_reg.predict_proba(X_test_scaled)[:, 1]
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    cm = confusion_matrix(y_test, y_pred)

    cv_scores.append(acc)
    cv_auc_scores.append(auc)
    cv_confusion_matrices.append(cm)
    print(f"Fold {i + 1} split: {len(train_idx)} training samples, {len(test_idx)} testing samples")
    print(f"Percentage split: {len(train_idx) / len(X) * 100:.1f}% train, {len(test_idx) / len(X) * 100:.1f}% test")


print(f"\nCross-validation results:")
print(f"Accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
print(f"AUC: {np.mean(cv_auc_scores):.4f} ± {np.std(cv_auc_scores):.4f}")
# Average confusion matrix across folds
avg_cm = np.mean(cv_confusion_matrices, axis=0)
print("\nAverage Confusion Matrix:")
print(avg_cm)



Performing lake-based cross-validation...
Fold 1 split: 512926 training samples, 128218 testing samples
Percentage split: 80.0% train, 20.0% test
Fold 2 split: 512900 training samples, 128244 testing samples
Percentage split: 80.0% train, 20.0% test
Fold 3 split: 512896 training samples, 128248 testing samples
Percentage split: 80.0% train, 20.0% test
Fold 4 split: 512928 training samples, 128216 testing samples
Percentage split: 80.0% train, 20.0% test
Fold 5 split: 512926 training samples, 128218 testing samples
Percentage split: 80.0% train, 20.0% test

Cross-validation results:
Accuracy: 0.8010 ± 0.0046
AUC: 0.8780 ± 0.0038

Average Confusion Matrix:
[[66750.8  9976. ]
 [15540.4 35961.6]]


## Model Training and Testing: Time-based Holdouts

In [15]:
print("\nTraining final model...")
X_all_scaled = scaler.fit_transform(X) # this is the training data
lake_ice_model = LogisticRegression(max_iter=100, random_state=42)
lake_ice_model.fit(X_all_scaled, y)

# Examine coefficients
feature_names = ['elevation', 'latitude', 'area', 'afdd', '$ T_{wk,L} $']
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lake_ice_model.coef_[0],
    'Abs_Coefficient': np.abs(lake_ice_model.coef_[0])
})
coefficients = coefficients.sort_values('Abs_Coefficient', ascending=False)
print("\nModel Coefficients (sorted by importance):")
print(coefficients)

# Create a time-based holdout set for final validation (optional but recommended)
X_time_train = X[ice_df['winter_year'] < 2022]
y_time_train = y[ice_df['winter_year'] < 2022]
X_time_test = X[ice_df['winter_year'] >= 2022]
y_time_test = y[ice_df['winter_year'] >= 2022]

X_time_train_scaled = scaler.fit_transform(X_time_train)
X_time_test_scaled = scaler.transform(X_time_test)

time_model = LogisticRegression(max_iter=1000, random_state=42)
time_model.fit(X_time_train_scaled, y_time_train)

y_time_pred = time_model.predict(X_time_test_scaled)
y_time_prob = time_model.predict_proba(X_time_test_scaled)[:, 1]



Training final model on all data...

Model Coefficients (sorted by importance):
        Feature  Coefficient  Abs_Coefficient
4  $ T_{wk,L} $    -1.401267         1.401267
1      latitude     0.900415         0.900415
3          afdd     0.721896         0.721896
0     elevation     0.645643         0.645643
2          area    -0.075735         0.075735


## The following 2 cells run the model on a specific lake and winter season to visualize performance. This is useful for model diagnostics and understanding how well the model captures seasonal ice dynamics.

In [31]:
# # Choose a specific lake and winter season
# lake_id = complete_df['lake_id'].unique()[0]  # You can change to any lake ID
# winter_year = 2022  # Choose a year with good observation coverage
#
# # Filter data for the selected lake and winter
# lake_season = complete_df[(complete_df['lake_id'] == lake_id) &
#                           (complete_df['winter_year'] == winter_year)].copy()
#
# # Sort by winter_doy
# lake_season = lake_season.sort_values('winter_doy')
#
# # Create dates for x-axis (convert winter_doy to actual dates)
# start_date = pd.Timestamp(f'{winter_year}-10-01')  # Winter day 0 = October 1
# lake_season['date'] = lake_season['winter_doy'].apply(
#     lambda x: start_date + pd.Timedelta(days=int(x)))
#
# # Create a new column for model predictions on ALL days (including observation days)
# # We need to re-run predictions on observation days
# observed_rows = lake_season[lake_season['data_type'] == 'Observed']
#
# # Extract features for all rows (both observed and unobserved)
# X_features = lake_season[['elevation', 'latitude', 'area', 'afdd', '$ T_{wk,L} $']]
# valid_rows = ~X_features.isna().any(axis=1)
# X_valid = X_features[valid_rows]
#
# # Scale and make predictions
# X_scaled = scaler.transform(X_valid)
# all_predictions = lake_ice_model.predict_proba(X_scaled)[:, 1]
#
# # Store predictions in a new column
# lake_season.loc[valid_rows, 'true_model_prediction'] = all_predictions
#


In [1]:
# # Create the figure
# plt.figure(figsize=(12, 6))
# ax = plt.gca()  # Get current axes
# ax.set_facecolor('w')
# ax.spines['left'].set_visible(True)
# ax.spines['left'].set_linewidth(1.0)
# ax.spines['left'].set_color('black')
# ax.spines['bottom'].set_visible(True)
# ax.spines['bottom'].set_linewidth(1.0)
# ax.spines['bottom'].set_color('black')
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# ax.tick_params(axis='both', which='major', direction='inout', width = 2, length = 10, left=True, bottom = True, labelsize=16)
# # Plot model predictions as a continuous line
# plt.plot(lake_season['date'], lake_season['true_model_prediction'],
#          color='k', linestyle='-', linewidth=2, label='Model Prediction')
#
# # Plot actual observations as points
# observed = lake_season[lake_season['data_type'] == 'Observed']
# plt.scatter(observed['date'], observed['ice_binary'],
#             color='k', marker = 'x', s=42, label='Observations', zorder=5)
#
# # Add threshold line
# plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Decision Threshold (0.5)')
#
# # Formatting
# # plt.title(f'Ice Model Performance - Lake ID: {lake_id}, Winter {winter_year}/{winter_year+1}')
# plt.ylabel('Ice Probability', fontsize=16)
# plt.ylim(-0.03, 1.03)
# plt.yticks([0, 0.5, 1], fontsize=14)
# # Format x-axis with month labels
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))
# plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
# plt.gcf().autofmt_xdate()
#
# # Add legend
# plt.legend(fontsize=16, loc='lower center')
# plt.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.show()

## Run the Model: prediction

In [25]:
# Prepare data for prediction (only for rows where ice_binary is None)
prediction_rows = complete_df['ice_binary'].isna()
print(f"Total rows needing prediction: {prediction_rows.sum()}")
# Extract features for prediction
X_to_predict = complete_df.loc[
    prediction_rows, ['elevation', 'latitude', 'area', 'afdd', '$ T_{wk,L} $']]
# Check for and drop rows with NaN values
rows_with_nans = X_to_predict.isna().any(axis=1)
if rows_with_nans.any():
    nan_count = rows_with_nans.sum()
    print(f"Dropping {nan_count} rows with NaN values ({nan_count / len(X_to_predict):.2%} of prediction set)")
    # Create a clean version with no NaNs
    X_to_predict_clean = X_to_predict.dropna()
    # Update our prediction_rows mask to exclude rows we're dropping
    valid_prediction_indices = X_to_predict_clean.index
    prediction_rows_clean = prediction_rows.copy()
    prediction_rows_clean[~prediction_rows_clean] = False  # Keep False values as False
    prediction_rows_clean[prediction_rows] = prediction_rows.index.isin(valid_prediction_indices)  # Update True values
    # Scale and predict only on clean data
    scalar = StandardScaler()
    X_scaled = scaler.transform(X_to_predict_clean)
    print(f"Predicting ice status for {len(X_scaled)} rows...")
    predictions = lake_ice_model.predict_proba(X_scaled)[:, 1]

    # Assign predictions only to the clean subset of rows
    complete_df.loc[valid_prediction_indices, 'ice_probability'] = predictions
    complete_df.loc[valid_prediction_indices, 'ice_prediction'] = (predictions > 0.5).astype(int)
else:
    # No NaNs found, proceed with all rows
    X_scaled = scaler.transform(X_to_predict)
    print(f"Predicting ice status for all {len(X_scaled)} rows...")
    predictions = lake_ice_model.predict_proba(X_scaled)[:, 1]
    complete_df.loc[prediction_rows, 'ice_probability'] = predictions
    complete_df.loc[prediction_rows, 'ice_prediction'] = (predictions > 0.5).astype(int)

# For observed data points, copy ice_binary to ice_prediction and set probability accordingly
observed_rows = ~prediction_rows  # "equals not the prediction rows..."
complete_df.loc[observed_rows, 'ice_prediction'] = complete_df.loc[observed_rows, 'ice_binary']
complete_df.loc[observed_rows, 'ice_probability'] = complete_df.loc[observed_rows, 'ice_binary']

# Fill any remaining NaN values in the prediction columns
missing_predictions = complete_df['ice_prediction'].isna()
if missing_predictions.any():
    print(f"Note: {missing_predictions.sum()} rows still have missing predictions (dropped due to NaNs)")
    # You can choose to fill these with a default value or leave them as NaN
    complete_df.loc[missing_predictions, 'ice_prediction'] = -1  # Special code for "couldn't predict"
    complete_df.loc[missing_predictions, 'ice_probability'] = -1


Total rows needing prediction: 7012240
Dropping 1881 rows with NaN values (0.03% of prediction set)
Predicting ice status for 7010359 rows...
Note: 1881 rows still have missing predictions (dropped due to NaNs)


## Generate Ice Metrics Dataset for visualization

In [26]:
# Calculate Ice Phenology Metrics with Confidence Scores
def calculate_ice_metrics(df, prob_threshold=0.5):
    """Calculates ice-on, ice-off, duration, and observation density for each lake and winter.
    Args:
        df (pd.DataFrame): DataFrame with ice_prediction, ice_probability, lake_id, winter_year, winter_doy, and data_type columns.
        prob_threshold (float): Probability threshold for determining ice presence.
    Returns:
        pd.DataFrame: DataFrame with ice metrics (ice_on_doy, ice_off_doy, ice_duration, obs_density).
    """
    # Filter ice data based on the probability threshold
    ice_data = df[df['ice_probability'] >= prob_threshold].copy()
    # Group by lake and winter to find ice-on and ice-off dates
    ice_on = ice_data.groupby(['lake_id', 'winter_year'])[
        'winter_doy'].min().reset_index()
    ice_off = ice_data.groupby(['lake_id', 'winter_year'])[
        'winter_doy'].max().reset_index()

    # Count number of days classified as ice for each lake and winter year
    ice_duration = ice_data.groupby(['lake_id', 'winter_year']).size().reset_index(name='ice_duration')
    # Merge ice-on and ice-off dates
    ice_metrics = pd.merge(ice_on, ice_off, on=['lake_id', 'winter_year'],
                           suffixes=('_on', '_off'))
    # Add ice duration to metrics
    ice_metrics = pd.merge(ice_metrics, ice_duration, on=['lake_id', 'winter_year'])
    # Calculate observation density
    total_days = df.groupby(['lake_id', 'winter_year'])[
        'winter_doy'].count().reset_index()
    observed_days = df[df['data_type'] == 'Observed'].groupby(['lake_id', 'winter_year'])[
        'winter_doy'].count().reset_index()
    # Merge total_days and observed_days
    obs_density = pd.merge(total_days, observed_days, on=[
        'lake_id', 'winter_year'], suffixes=('_total', '_observed'), how='left')
    obs_density['obs_density'] = obs_density['winter_doy_observed'].fillna(
        0) / obs_density['winter_doy_total']
    # Merge ice metrics with observation density
    ice_metrics = pd.merge(ice_metrics, obs_density[[
        'lake_id', 'winter_year', 'obs_density']], on=['lake_id', 'winter_year'])
    # Calculate confidence score
    ice_metrics['confidence'] = ice_metrics['obs_density']

    return ice_metrics
# Apply the function to calculate ice metrics
################### RESULTS ###################################
ice_metrics_df = calculate_ice_metrics(complete_df)
###############################################################

In [27]:
ice_metrics_df.to_csv(r'USER_INPUT', index=False) # save the output CSV